In [42]:
import warnings
warnings.filterwarnings("ignore")

In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
df = pd.read_csv("card.csv", header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
df.shape

(284807, 31)

In [7]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
dtype: int64

In [14]:
df[30].value_counts()

0    284315
1       492
Name: 30, dtype: int64

In [20]:
fraud_df = df[df[30] == 1]
fraud_df.shape

(492, 31)

In [29]:
X = df[range(30)].values
y = df[30].values

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [41]:
# checking stratified
np.unique(y_val, return_counts=True)[1][1] / np.unique(y_val, return_counts=True)[1][0] * 100

0.17266371077762618

In [45]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [46]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), verbose=2)

Epoch 1/20
5964/5964 - 14s - loss: 10.9562 - accuracy: 0.9962 - val_loss: 4.8397 - val_accuracy: 0.9983 - 14s/epoch - 2ms/step
Epoch 2/20
5964/5964 - 13s - loss: 5.3007 - accuracy: 0.9964 - val_loss: 3.0371 - val_accuracy: 0.9983 - 13s/epoch - 2ms/step
Epoch 3/20
5964/5964 - 13s - loss: 1.8058 - accuracy: 0.9965 - val_loss: 0.4999 - val_accuracy: 0.9983 - 13s/epoch - 2ms/step
Epoch 4/20
5964/5964 - 11s - loss: 0.4706 - accuracy: 0.9973 - val_loss: 0.0238 - val_accuracy: 0.9987 - 11s/epoch - 2ms/step
Epoch 5/20
5964/5964 - 12s - loss: 0.1290 - accuracy: 0.9977 - val_loss: 0.0205 - val_accuracy: 0.9983 - 12s/epoch - 2ms/step
Epoch 6/20
5964/5964 - 13s - loss: 0.0242 - accuracy: 0.9983 - val_loss: 0.0172 - val_accuracy: 0.9983 - 13s/epoch - 2ms/step
Epoch 7/20
5964/5964 - 11s - loss: 0.0258 - accuracy: 0.9982 - val_loss: 0.0128 - val_accuracy: 0.9983 - 11s/epoch - 2ms/step
Epoch 8/20
5964/5964 - 12s - loss: 0.0154 - accuracy: 0.9982 - val_loss: 0.0127 - val_accuracy: 0.9983 - 12s/epoch - 

In [55]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

1469/1469 [==============================] - 1s 934us/step


In [56]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9982763757075371

In [49]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

1469/1469 [==============================] - 1s 697us/step - loss: 0.3419 - accuracy: 0.9983
Test Accuracy: 0.9982763528823853


In [50]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]